In [72]:
import json
import pandas as pd

In [73]:
ref_path="database/output/refs0926.json"

with open(ref_path,"r") as f:
    refs=json.load(f)

In [74]:
lis_list=[r["Lisence"] for r in refs]

In [75]:
set(lis_list)

{'Creative Commons Attribution (CC BY) licence to any author accepted manuscript version arising from this submission Disclaimer: The views expressed are those of the authors and not necessarily those of the United Kingdom (UK) Department of Health and Social Care, the National Health Service, the National Institute for Health and Care Research (NIHR), the UK Health Security Agency, the World Health Organization or of the commentators."- - Authors and Affiliations - "MRC Centre for Global Infectious Disease Analysis & WHO Collaborating Centre for Infectious Disease Modelling, Jameel Institute, Imperial College London, London, UKDavid J. Haw,_Giovanni Forchini,_Patrick Doohan,_Paula Christen,_ Matteo Pianella,_Robert Johnson,_Alexandra B. Hogan,_Peter Winskill,_Peter J. White,_Azra C. Ghani,_Neil M. Ferguson_&_Katharina D. HauckUSBE, UmexE5 Universitet, UmexE5, SwedenGiovanni ForchiniDepartment of Zoology, University of Oxford, Oxford, UKSumali BajajSchool of Population Health, Universi

In [76]:
allow_lisences=[
 'Creative Commons Attribution 3.0 Unported Licence. ',
 'Creative Commons Attribution 3.0 Unported License. ',
 'Creative Commons Attribution Noncommercial Share Alike 3.0 Unported Licence. ',
 'Creative Commons Attribution- NonCommercial-ShareALike 3.0 Unported License. ',
 'Creative Commons Attribution-Non-Commercial-ShareAlike 3.0 Unported licence. ',
 'Creative Commons Attribution-NonCommercial- ShareALike 3.0 Unported License. ',
 'Creative Commons Attribution-NonCommercial-Share Alike 3.0 License. ',
 'Creative Commons Attribution-NonCommercial-Share Alike 3.0 Unported License. ',
 'Creative Commons Attribution-NonCommercial-ShareALike 3.0 Unported License. ',
 'Creative Commons Attribution-NonCommercial-ShareALike 3.0 UnportedLicense. ',
 'Creative Commons Attribution-NonCommercial-ShareAlike 3.0 Unported License. ',
 'Creative Commons Attribution-NonCommercial-ShareAlike 3.0 Unported licence. ',
 'Creative Commons Attribution-NonCommercial-ShareAlike 3.0 Unported license. ',

]

In [77]:
sel_refs=[r for r in refs if r["Lisence"] in allow_lisences]

In [78]:
len(sel_refs)

1972

In [79]:

with open("database/output/1019translatable_refs.json","w") as f:
    refs=json.dump(sel_refs,f,indent=1)

In [80]:
i=0
context_list=[]
for i in range(len(sel_refs)):
    text_list=sel_refs[i]["main"][0]
    for text in text_list:
        text=text.strip()
        if len(text)<20:
            continue

        d={
            "text":text,
            "license":sel_refs[i]["Lisence"].strip(),
            "doi":sel_refs[i]["doi"],

        }
        context_list.append(d)


len(context_list)

7828

In [81]:

with open("database/output/1019translatable_context_dict.json","w") as f:
    refs=json.dump(context_list,f,indent=1)

In [86]:
with open("database/output/1019translatable_context.json","w") as f:
    refs=json.dump([r["text"] for r in context_list],f,indent=1)

In [95]:
#translate
import requests
#deepl
API_KEY=input()
#PI_KEY = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxx' # 自身の API キーを指定

def translate(text ,source_lang = 'EN',target_lang = 'JA'):

    # パラメータの指定
    params = {
                'auth_key' : API_KEY,
                'text' : text,
                'source_lang' : source_lang, # 翻訳対象の言語
                "target_lang": target_lang  # 翻訳後の言語
            }

    # リクエストを投げる
    request = requests.post("https://api-free.deepl.com/v2/translate", data=params) # URIは有償版, 無償版で異なるため要注意
    result = request.json()

    translated_text=result["translations"][0]["text"]
    return translated_text


In [96]:
"""
#textra
import requests as req
from requests_oauthlib import OAuth1
from xml.etree.ElementTree import *

NAME = 'john20231020'
KEY = ''
SECRET = ''
URL = 'https://mt-auto-minhon-mlt.ucri.jgn-x.jp/api/mt/c-17566_en_de/'

consumer = OAuth1(KEY , SECRET)

def translate(text):
    params = {
        'key': KEY,
        'name': NAME,
        "type":"json",
        "text":text,
    }    

    res = req.post(URL , data=params , auth=consumer)
    res.encoding = 'utf-8'
    trans_text=json.loads(res.text)["resultset"]["result"]["information"]["text-t"]

    return trans_text
"""

'\n#textra\nimport requests as req\nfrom requests_oauthlib import OAuth1\nfrom xml.etree.ElementTree import *\n\nNAME = \'john20231020\'\nKEY = \'7615f08cad124bec08b1771b2a10251d06531ed06\'\nSECRET = \'fe46b9a132183bde292fe65abf5b1cf3\'\nURL = \'https://mt-auto-minhon-mlt.ucri.jgn-x.jp/api/mt/c-17566_en_de/\'\n\nconsumer = OAuth1(KEY , SECRET)\n\ndef translate(text):\n    params = {\n        \'key\': KEY,\n        \'name\': NAME,\n        "type":"json",\n        "text":text,\n    }    \n\n    res = req.post(URL , data=params , auth=consumer)\n    res.encoding = \'utf-8\'\n    trans_text=json.loads(res.text)["resultset"]["result"]["information"]["text-t"]\n\n    return trans_text\n'

In [99]:

trans_dict={}

target_lang="DE"
dict_path=f"database/output/1020translatable_context_eng_to_ger_dict_{target_lang}.json"

with open(dict_path,"r") as f:
    trans_dict=json.load(f)

In [101]:
from tqdm import tqdm
import random
import time

random.shuffle(context_list)

i=0

def save():
    with open(dict_path,"w") as f:
        json.dump(trans_dict,f,indent=1)

for i in tqdm(range(len(context_list))):
    eng_text=context_list[i]["text"]

    if eng_text not in trans_dict:
        try:
            #ger_text=translate(eng_text)
            ger_text=translate(eng_text,target_lang=target_lang)
            trans_dict[eng_text]=ger_text
        except Exception as e:
            print("error!",e,i)
            time.sleep(10)
        #print(ger_text)
    if i%20==0:
        save()
save()

  1%|▏         | 100/7828 [10:46<13:52:08,  6.46s/it]


KeyboardInterrupt: 

In [113]:
l=[]
l.extend([r["text"] for r in context_list])
l.extend(trans_dict.values())
random.shuffle(l)
with open("database/output/1019translatable_EN_plus_part_GE.json","w") as f:
    refs=json.dump(l,f,indent=1)